 
 # International Training Course
 
 # Scenario Loss Exercise
 
 ### mhaas@gfz-potsdam.de
 
 
 
 

## language: Python 2.7

### required packages


In [ ]:
import math
import numpy
import scipy.stats
import pandas
import sqlalchemy
import matplotlib.pyplot as plt
import warnings

%matplotlib inline

 # 1. Exposure
 
 ## Tasks
    - Connecting to the database
    - Plotting a distribution of the material type
    - Plotting age of the buildings


### Collecting data
Now let's connect to the database and get all analyzed buildings

In [ ]:
# database connection
user = 'user'
password = 'password'
host = 'host'

engine = sqlalchemy.create_engine('postgresql://{}:{}@{}:5432/rem'.format(user,password,host))

survey_id = 8

# query to the database
def get_proc_expo(engine,tablename,survey_id):
    
    sql = """
    SELECT 
    st_x(st_transform(st_centroid(v.the_geom),4326)) lon,
    st_y(st_transform(st_centroid(v.the_geom),4326)) lat,
    v.gid,
    v.mat_type,
    v.year_1 built,
    v.height_1 floors,
    v.occupy
    FROM {} v WHERE survey_gid = {} AND rrvs_status='UNMODIFIED';""".format(tablename,survey_id)

    t=pandas.read_sql_query(sql,con=engine)
    return t

#store data in pandas dataframe
data = get_proc_expo(engine,'asset.ve_object',survey_id)

### Inspect the collected data

In [ ]:
data

In [ ]:
# We can investigate the individual attributes and also plot them
variable = 'mat_type'

#select only the column and get the counts (to get normalized values devide by the sum)
data[variable].value_counts()#/sum(data[variable].value_counts())

In [ ]:
# Plot the distribution
plt.figure()
data[variable].value_counts().plot.bar()
plt.show()

# 2. Fragility and Vulnerability 
 
 ## Tasks
    - Fragility curves for vulnerability classes
    - Assigning EMS-98 building types
    - Fragility curves for building types
    - Plotting curves for different building types

### Fragility models for EMS-98 vulnerabilty classes

First take a look at the fragility models as defined for each vulnerability class in Giovinazzi and Lagomarsino (2005).

![ems98_vc.jpeg](ems98_vc.jpeg)

NOTE: We only use the most likely vulnerability index values here.

In [ ]:
# Routine to create a discrete fragility function for OpenQuake
# i.e. f(MMI,DG)=P(dg > DG|Building type)
# as well as a discrete vulnerability function
# using the Giovinazzi vulnerability index method
import math
import scipy.stats
import numpy

#levels of ground motion (as EMS-98 macroseismic intensity)
#III-XI
gms = [3.+i*1. for i in range(10)]

#Vulnerabilty classes and vulnerability indices
classes = ['A','B','C','D','E','F']
vuln_indexes = [0.9,0.74,0.58,0.42,0.26,0.1]

#Generate a data frame that will hold the models
fragility = pandas.DataFrame()

In [ ]:
fragility['VulnClass']     = numpy.repeat(classes,len(gms))
fragility['VulnIndex']     = numpy.repeat(vuln_indexes,len(gms))
fragility['EMS-98'] = gms*len(classes)

In [ ]:
# Calculate mean damage grade mu_d for each ground motion step in gm
fragility['mu_d'] = [2.5*(1+math.tanh(v)) for v in (fragility['EMS-98'] + 6.25*fragility['VulnIndex'] - 13.1)/2.3]

In [ ]:
def pbeta(mu_d,t=8):
    '''
    Function to calculate damage grade distribution (beta distribution with a shape close to lognormal t=8)
    input: mu_d - mean damage grade
           t    - shape parameter of the beta distribution,default t=8
    ouput: returns a list of scipy.stats.beta objects, one for each damage grade in mu_d
    '''
    #bounds of dg0-5 (upper bound = x+1 --> 6)
    a,b = 0,5
    #r,t parameters according to Giovinazzi 2005 (t=8)
    r = t * (0.007 * mu_d**3 - 0.0525 * mu_d**2 + 0.2875 * mu_d)

    #convert r,t to alpha,beta
    alpha = r
    beta = t - alpha
    return scipy.stats.beta(alpha,beta,loc=a,scale=b-a)

In [ ]:
#Get beta distribution for each groundmotion and Vulnerability Index from mean damage grade
fragility['beta_dist'] = [pbeta(mu_d) for mu_d in fragility['mu_d']]


In [ ]:
#Plot PDF's for each gm step for a given VC
building_type = 'A'

#get a set of damage grade values (and intermediate for pdf)
dgs = [0.01*i for i in range(1,int(max(gms)*100)+1)]

#show distributions
samples=[]
for gm in fragility['EMS-98'].unique():
    #sample distribution
    samples.append(fragility.loc[(fragility['VulnClass']==building_type)&(fragility['EMS-98']==gm),'beta_dist'].iloc[0].rvs(1000))

#plot boxplot
plt.boxplot(samples,positions=fragility['EMS-98'].unique())
plt.xlabel('Macroseismic intensity')
plt.ylabel('Damage grade')
plt.show()

In [ ]:
#get probability for each damage grade
#NOTE: We assume that dg0.5 till dg1.5 --> dg1 etc.
for dg in range(5):
    fragility['dg{}'.format(dg)] = [v.cdf(dg+0.5)-v.cdf(dg-0.5) for v in fragility['beta_dist']]

#last damage grade is 1-sum others
fragility['dg5']=1.-fragility[['dg0','dg1','dg2','dg3','dg4']].sum(axis=1)
fragility

## Assigning fragility models to the buildings

Let's try to assign some vulnerability classes to the buildings we have analysed in the following, 
of course this can only be done in a very simplified way in the framework of this workshop.

First try to find a EMS-98 building type that may be described with the material types that you found
Try to map the material types to the building types below.

![ems98_types.jpeg](ems98_types.jpeg)

In [ ]:
#Find unique material types in our dataset
data['mat_type'].unique()

In [ ]:
#Define a building type for each material type in the dataset
mat_types = ['MUR','CR']
bdg_types = ['M1','RC1']

#Find the coresponding most likely vulnerability index NOTE: use same order as above for mat_types!
vuln_index_ml = [0.873,0.644]

#Find min and max values
vun_index_min = [0.62,0.3]
vuln_index_max = [1.02,1.02]

In [ ]:
#store as a dataframe and select only one VI for now!

#WHICH VI should be used?
vuln_index= vuln_index_ml

#store
my_fragility = pandas.DataFrame()
my_fragility['mat_type'] = numpy.repeat(mat_types,len(gms))
my_fragility['bdg_type'] = numpy.repeat(bdg_types,len(gms))
my_fragility['EMS-98'] = gms*len(mat_types)
my_fragility['VulnIndex'] = numpy.repeat(vuln_index,len(gms))

#!!! CHECK THE DATAFRAME IF NOT ALL MAT_TYPES HAVE A MAPPING THE FOLLOWING SCRIPTS WILL FAIL!!!
my_fragility

## Building type assignment
Assign each building a building type make sure every type is defined!

In [ ]:
#ONLY SUBSET
#data=data.loc[(data['mat_type']=='CR')|(data['mat_type']=='MUR')].copy()
#data=data.reset_index()
#data['mat_type'].unique()

In [ ]:
try:
    data['bdg_type']= [my_fragility.loc[my_fragility['mat_type']==mat_type,'bdg_type'].iloc[0] for mat_type in data['mat_type']]
except:
    print 'There are undefined building type mappings!'
    raise Exception


In [ ]:
#Check the building types
data['bdg_type'].unique()

 ### Fragility curves
 
 We can now apply the same approach as for the individual vulnerability classes before 
 and assign each building with a probability distribution

In [ ]:
# Calculate mean damage grade mu_d for each ground motion step in gm
my_fragility['mu_d'] = [2.5*(1+math.tanh(v)) for v in (my_fragility['EMS-98'] + 6.25*my_fragility['VulnIndex'] - 13.1)/2.3]

#Get beta distribution for each groundmotion and Vulnerability Index from mean damage grade
my_fragility['beta_dist'] = [pbeta(mu_d) for mu_d in my_fragility['mu_d']]

#get probability for each damage grade
#NOTE: We assume that dg0.5 till dg1.5 --> dg1 etc.
for dg in range(5):
    my_fragility['dg{}'.format(dg)] = [v.cdf(dg+0.5)-v.cdf(dg-0.5) for v in my_fragility['beta_dist']]

#last damage grade is 1-sum others
my_fragility['dg5']=1.-my_fragility[['dg0','dg1','dg2','dg3','dg4']].sum(axis=1)
my_fragility

In [ ]:
#Plot the fragility functions for a given building type
bdg_type='RC1'

#NOTE: We have the PDF representation here not the more common CDF representation!
#plot it
legend=[]
for dg in range(6):
    x = my_fragility.loc[my_fragility['bdg_type']==bdg_type,'EMS-98']
    y = my_fragility.loc[my_fragility['bdg_type']==bdg_type,'dg{}'.format(dg)]
    plt.plot(x,y)
    legend.append('dg{}'.format(dg))
plt.legend(legend)
plt.show()

# 3. Vulnerability
 
 ## Tasks
    - Get vulnerability curve for mean loss ratios from fragility curves
      and a loss transfer function
    - Plotting vulnerability for different building types
    - Assigning a value to the buildings



### Loss transfer function

Now we should have a fragility model for each of the buildings in our data set
, i.e., for each macroseismic intensity we have a probability to reach 
a specific damage state.

Now considering a damage to loss transfer function as defined e.g. in Moroux 2004:

|EMS-98 damage grade | loss ratio % |
|:-----------|------------:|
|DG1 | 2|
|DG2 |10|
|DG3 |50|
|DG4 |100|
|DG5 |100|


We can use this information to calculate a mean vulnerability curve from these values
as:

$P(MeanLossRatio|intensity) = \sum_{DG_i;i=1,5} LossRatio_i*p(DG_i|gm)$



In [ ]:
#vulnerability
vulnerability = my_fragility[['bdg_type','EMS-98','dg0','dg1','dg2','dg3','dg4','dg5']].copy(deep=True)

#loss ratios dg1...dg5
loss_ratios = [0,0.02,0.10,0.50,1.00,1.00]
#Get vulnerability
#vulnerability['LR|I']=0
vulnerability[['dg0','dg1','dg2','dg3','dg4','dg5']]=vulnerability[['dg0','dg1','dg2','dg3','dg4','dg5']]*loss_ratios
vulnerability['Loss Ratio (I)']=vulnerability[['dg0','dg1','dg2','dg3','dg4','dg5']].sum(axis=1)
vulnerability = vulnerability.drop(['dg0','dg1','dg2','dg3','dg4','dg5'],1)
vulnerability

In [ ]:
#Plot the vulnerability function for a given building type
bdg_type='RC1'

#plot it
x = vulnerability.loc[vulnerability['bdg_type']==bdg_type,'EMS-98']
y = vulnerability.loc[vulnerability['bdg_type']==bdg_type,'Loss Ratio (I)'.format(dg)]
plt.plot(x,y)
plt.legend([bdg_type])
plt.xlabel('Macroseismic intensity')
plt.ylabel('Mean loss ratio')
plt.show()

## Assign value

In order to calculate an earthquake loss scenario we need to assign a value to each building.
We assign each storey of a building a value of 100 tsdUSD.

First lets set the buildings with unknown height to NA and then for the rest estimate a value
from the height.


In [ ]:
#unknown 99 to nan
data.loc[data['floors']==99.] = numpy.nan

#each floor is 100 tsd USD
data['tsdUSD'] = data['floors']*100

#show total value
print 'Total assigned exposure value [tsd USD]: {}'.format(data['tsdUSD'].sum())

In [ ]:
#There might be rows with NaN now remove them
data.loc[data.isnull().any(axis=1)]
#remove in case
data=data.dropna()
data.reset_index()


 
 ## 4. Loss Scenario
 
 ### Tasks
     - Define an earthquake scenario and plot it
     - Calculate the macroseismic intensity using a simple IPE
     - Plot the ground motion field
     - Estimate the loss for each building
     - Investigate the loss distribution

Finally we can calculate a scenario loss for our exposure.

We define a simple Intensity Prediction Equation:

In [ ]:
def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(math.radians, [lon1, lat1, lon2, lat2])
    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.asin(math.sqrt(a)) 
    km = 6367 * c
    return km

def IPE(lon_sites,lat_sites,lon_eq,lat_eq,mw,z):
    # Calculates the intensities resulting from an event at given site locations
    # implemented from Allen, Wald & Worden 2012
    # only the hypocentral distance model was implemented (not the rupture distance model)
    # input: 1) mw = Moment magnitude (Scalar)
    #        2) lon_sites/lat_sites = locations to calculate intensity at
    #        3) lon_eq/lat_eq = epicentre
    #        4) depth = hypocentral depth [km]
    # output:1) dataframe containing magnitude,intensity[MMI],sigma[MMI],epicentral distance [km]
  
    #Parameters for IPE
    c0 =  2.085
    c1 =  1.428
    c2 = -1.402
    c4 =  0.078
    m1 = -0.209
    m2 =  2.042
    #Parameters for distance dependent uncertainty
    s1 =  0.82
    s2 =  0.37
    s3 = 22.9
    
    #Calculate hypocentral distances
    R = [haversine(lon_eq,lat_eq,x,y) for x,y in zip(lon_sites,lat_sites)]
  
    #Calculate attenuation
    #Rm
    Rm = m1 + m2 * math.exp(mw-5)
    
    #close site effect
    switchON = [1 if r>50 else 0 for r in R]
    
    #return intensities and sigma
    intensity = [c0 + c1*mw + c2*math.log(math.sqrt(r**2+Rm**2))+sw*c4*math.log(r/50.) for r,sw in zip(R,switchON)]
    #sigmas = [s1 + s2/(1+(r/s3)^2) for r in R]
  
    return intensity#,sigmas

In [ ]:
# Define a scenario earthquake, simple point source

event_mw = 4.5 # moment magnitude of the earthquake 
event_lon = 7.9 # longitude of the epicentre
event_lat = 48.9 # latitude of the epicentre
event_depth = 10 # focal depth in Km

In [ ]:
#plot the scenario
plt.scatter(data['lon'],data['lat'],color='blue') # plot the locations of the buildings
plt.scatter(event_lon,event_lat,marker='*',s=math.exp(event_mw),color='red') # plot the earthquake´s epicentre
plt.show()

In [ ]:
# Calculate ground motion for each building

data['gm'] = IPE(data['lon'],data['lat'],event_lon,event_lat,event_mw,event_depth)

In [ ]:
#take non-nan
subset = data[['lon','lat','gm']].dropna()

#how many
n = min(len(data['lon']),300)
x = subset['lon'].iloc[0:n]
y = subset['lat'].iloc[0:n]
z = subset['gm'].iloc[0:n]

#plot
CS = plt.tricontourf(x,y,z,levels=[1,2,3,4,5,6,7,8,9,10,11,12],cmap=plt.cm.jet)
plt.colorbar(CS)
plt.scatter(event_lon,event_lat,marker='*',s=math.exp(event_mw),color='red')
plt.show()

### Now we have all ingredients just calculate the loss for each building


In [ ]:
data['loss'] = [vulnerability.loc[
    (vulnerability['bdg_type']==data['bdg_type'].iloc[i])&
    (vulnerability['EMS-98']==round(data['gm'].iloc[i])),'Loss Ratio (I)'].iloc[0]*
                data['tsdUSD'].iloc[i] for i in range(len(data['gm']))]

### Let's take a look at the loss distribution


In [ ]:
#Total loss
data['loss'].sum()

In [ ]:
#Loss aggregated per building type
data.groupby('bdg_type')['loss'].sum()

In [ ]:
plt.hist(data['loss'])
plt.show()

In [ ]:
#different building types
data['bdg_type'].unique()

In [ ]:
#Check individual building types
bdg_type='M1'
plt.hist(data['loss'].loc[data['bdg_type']==bdg_type])
plt.show()

In [ ]:
#Check individual building types
bdg_type='RC1'
plt.hist(data['loss'].loc[data['bdg_type']==bdg_type])
plt.show()